Needs 3 text files with data, dataset.py, dataloader.py, and models (feedforward.py)

In [1]:
! pip install pytorch_lightning
! pip install torchmetrics

     |████████████████████████████████| 527 kB 8.2 MB/s 
     |████████████████████████████████| 596 kB 62.8 MB/s 
     |████████████████████████████████| 952 kB 58.6 MB/s 
     |████████████████████████████████| 133 kB 80.9 MB/s 
     |████████████████████████████████| 397 kB 78.6 MB/s 
     |████████████████████████████████| 829 kB 63.6 MB/s 
     |████████████████████████████████| 1.1 MB 76.4 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 144 kB 84.6 MB/s 
     |████████████████████████████████| 271 kB 101.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=8543a1112a4bd766601bde1ddc63e43470a87fbb4ee460e6a4802202ea68b669
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfu

In [27]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from dataset import wiki_dataset
from dataloader import wiki_dataloader
import torchmetrics
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
  # Load datasets
  train = wiki_dataset('wiki.train.txt', training=True, token_map='create', window=30)
  valid = wiki_dataset('wiki.valid.txt', training=False, token_map=train.token_map, window=30)
  test = wiki_dataset('wiki.test.txt', training=False, token_map=train.token_map, window=30)
  datasets = [train,valid,test]

  # Load dataloader
  dataloader = wiki_dataloader(datasets=datasets, batch_size=20)


In [40]:
class rnn(pl.LightningModule):
    def __init__(self, n_vocab, embedding_size, hidden_size, num_layers):
        super(rnn, self).__init__()
        self.embed = nn.Embedding(n_vocab, embedding_size)
        nn.init.uniform_(self.embed.weight, a=-0.1, b=0.1)
        self.rnn = nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=False,dropout=0.2)
        nn.init.uniform_(self.rnn.weight_ih_l0, a=-0.1, b=0.1)
        nn.init.uniform_(self.rnn.weight_hh_l0, a=-0.1, b=0.1)
        nn.init.uniform_(self.rnn.weight_ih_l1, a=-0.1, b=0.1)
        nn.init.uniform_(self.rnn.weight_hh_l1, a=-0.1, b=0.1)
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.fc.weight = self.embed.weight
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.embed(x)
        x, hidden = self.rnn(x)
        x = x[:, -1, :]
        logits = self.fc(x)

        return logits

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=3e-3)

    def training_step(self, batch, batch_idx):
        data, label = batch
        logits = self.forward(data)
        loss = self.loss(logits, label)
        tensorboard_logs = {'loss': {'train': loss.detach()}}
        self.log("train loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        data, label = batch
        logits = self.forward(data)
        loss = self.loss(logits, label)
        tensorboard_logs = {'loss': {'val': loss.detach()}}
        self.log("val loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

    def test_step(self, batch, batch_idx):
        data, label = batch
        logits = self.forward(data)
        loss = self.loss(logits, label)
        tensorboard_logs = {'loss': {'test': loss.detach()}}
        self.log("test loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

In [ ]:
  model = rnn(n_vocab = len(train.unique_tokens), embedding_size=100, hidden_size=100, num_layers=2)
  tb_logger = pl_loggers.TensorBoardLogger("./lightning_logs/", name="ff")
  trainer = pl.Trainer(logger=tb_logger, max_epochs=20, gpus=1, callbacks=[EarlyStopping(monitor='val loss')])
  trainer.fit(model, dataloader)
  result = trainer.test(model, dataloader)
  print(result)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | embed | Embedding        | 2.9 M 
1 | rnn   | RNN              | 40.4 K
2 | fc    | Linear           | 2.9 M 
3 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.838    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

In [21]:
model.fc.weight is model.embed.weight

True